# Lista do site Ranking dos Políticos: https://www.politicos.org.br/

Para o site politicos.org é preciso usar a biblioteca Selenium, pois alguns site utilizam o conceito conhecido como single page application, algumas aplicações desse tipo só criam a página web do lado do navegador. Então é necessário que o navegador faça a requisição para que o Html seja montado no momento que usuario pede. Nesses casos o BeautifulSoup não é suficiente, então recorre-se ao Selenium que é um navegador automatizado, um webdriver.

Checkpoints sugeridos:

1) Coleta de dados via web scrapping. Bibliotecas como Selenium e BeautifulSoup podem ajudar.<br>
<br>
2) Análise exploratória: Gráficos de dispersão entre nota padronizada dos dois rankings. São correlacionados? Existem outliers?<br>
<br>
3) Modelo explicativo: Regressão linear múltipla entre os 4 itens do ranking do Legisla Brasil e a nota do Ranking dos Políticos, por exemplo. Os 4 itens do Legisla explicam qual variabilidade do Ranking dos Políticos?<br>
<br>
4) Discussão: Por que são diferentes? Por que são iguais? Existem viéses políticos? São apartidários? Qual sua opinião que justifique os resultados?<br>

In [2]:
from bs4 import BeautifulSoup
import requests 
import pandas as pd
import numpy as np
import re
from unidecode import unidecode #retirar acentos
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from time import sleep

In [3]:
# formatando a janela do chrome que irá ser usada na pesquisa do selenium
options = Options()
#options.add_argument('--headless') #utilizar quando não desejar que a página abra
options.add_argument('window-size=400,800')
options.page_load_strategy = 'eager'

In [4]:
# criando o driver

navegador = webdriver.Chrome(options=options)

navegador.get('https://www.politicos.org.br/Ranking')
sleep(2)

#click para aceitar os cookies
navegador.find_element(By.XPATH, '//*[@class="mb-3"]').click()

lista_notas = []

In [5]:
#localizando o botão senador
button_senador = navegador.find_elements(By.CSS_SELECTOR, "span > h6")[1]
# Clicar no botão para senador
button_senador.click()

In [6]:
#abrindo a página
a = 50
for n in range(a):
    navegador.find_element(By.XPATH, "/html/body/div/div[2]/div[2]/div/div/div/div[8]/div/div[2]/button").click()
    sleep(4)

In [7]:
# criando lista para notas e nomes
lista_final = []

In [8]:
#lista temporária
lista_nome_notas = []
# pegando os dados dos candidatos na pag totalmente aberta
html_informacoes = navegador.find_elements(By.CSS_SELECTOR, 'button > div > div > div > div > h2')
# gerando a lista de notas e nomes
for x in range(0, len(html_informacoes)):
    lista_nome_notas.append(html_informacoes[x].text)

In [9]:
# dividindo para cada candidato
for i in range(0, len(lista_nome_notas), 3):
    lista_final.append(lista_nome_notas[i:i + 3])

In [10]:
# formatando lista
for n in range(len(lista_final)):
    lista_final[n][0] = unidecode(lista_final[n][0]).lower()
    lista_final[n][2] = float(lista_final[n][2].replace(',', '.'))
    del lista_final[n][1]

In [11]:
df_site2 = pd.DataFrame(lista_final, columns = ['nome', 'nota'])

In [12]:
df_site2 = df_site2.sort_values(by=['nome'])
df_site2

,nome,nota
183,abou anni,6.55
280,acacio favacho,6.12
133,adolfo viana,6.82
1,adriana ventura,8.24
134,adriano do baldy,6.82
...,...,...
462,ze carlos,2.78
425,ze neto,3.20
207,ze silva,6.47
266,ze vitor,6.22


In [52]:
#salvando o df em arquivo .csv
df_site2.to_csv('df_site2.csv', index = False)

In [13]:
#importanto lista gerada para o site legislabr
df_site1 = pd.read_csv('rank_legislabr_completo.csv') 

In [14]:
# formatando
df_site1 = df_site1.drop(columns=['Unnamed: 0'])

In [15]:
df_site1

,nome,estado,partido,n_prod_leg,n_fisc,n_mobi,n_alin_partd
0,abilio santana,Bahia,PSC,1.7,0.0,0.9,9.1
1,abou anni,São Paulo,UNIÃO,3.8,0.9,2.4,6.4
2,acacio favacho,Amapá,MDB,2.6,0.2,2.1,6.4
3,adolfo viana,Bahia,PSDB,2.8,0.9,2.4,5.5
4,adriana ventura,São Paulo,NOVO,6.4,2.9,7.3,10.0
...,...,...,...,...,...,...,...
561,ze carlos,Maranhão,PT,3.1,2.0,6.4,7.3
562,ze neto,Bahia,PT,4.2,1.1,5.5,7.3
563,ze silva,Minas Gerais,SOLIDARIEDADE,4.3,0.5,3.6,6.4
564,ze vitor,Minas Gerais,PL,5.8,1.8,4.5,6.4


In [16]:
# Unindo as duas colunas
df_3 = pd.merge(df_site1, df_site2,  how='left')


In [53]:
# salvando o dataframe resultado da união dos dataframes dos sites buscados
df_3.to_csv('df_merge.csv', index = False)

In [49]:
# verificando as linhas diferentes entre os dois dataframes
candidatos_n_exercicio = pd.concat([df_site1['nome'],df_site2['nome']]).drop_duplicates(keep=False)

In [50]:
#candidatos não listados nos dois sites
list(candidatos_n_exercicio)

['abilio santana',
 'aguinaldo ribeiro',
 'alan rick',
 'alexandre serfiotis',
 'anibal gomes',
 'assis carvalho',
 'bia cavassa',
 'boca aberta',
 'darcisio perondi',
 'deuzinho filho',
 'dr. goncalo',
 'dr. joao',
 'dr. sinval malheiros',
 'edmilson rodrigues',
 'eduardo braide',
 'ely santos',
 'eneias reis',
 'fabiano tolentino',
 'fabio faria',
 'fabio mitidieri',
 'fernando borja',
 'flordelis',
 'gastao vieira',
 'gorete pereira',
 'henrique do paraiso',
 'joao h. campos',
 'jorge goetten',
 'jose nunes',
 'julio lopes',
 'laura carneiro',
 'leonardo gadelha',
 'leonardo picciani',
 'luiz flavio gomes',
 'manuel marcos',
 'marcao gomes',
 'marcelo brum',
 'marcelo moraes',
 'marcio jerry',
 'margarida salomao',
 'mariana carvalho',
 'miguel haddad',
 'moses rodrigues',
 'nelson pellegrino',
 'ney leprevost',
 'nilson f. stainsack',
 'norma pereira',
 'odorico monteiro',
 'osires damaso',
 'otavio leite',
 'paes landim',
 'patrick dorneles',
 'paulo marinho jr',
 'paulo vicente c

In [47]:
# encontrar linhas que não estão presentes nos dois dataframes
for x in df_site2.itertuples():
    if x[1].find('silva') != -1:
        print(x)

Pandas(Index=50, nome='ale silva', nota=7.39)
Pandas(Index=464, nome='benedita da silva', nota=2.78)
Pandas(Index=488, nome='orlando silva', nota=2.5)
Pandas(Index=258, nome='ossesio silva', nota=6.25)
Pandas(Index=235, nome='ricardo silva', nota=6.33)
Pandas(Index=207, nome='ze silva', nota=6.47)
